BPnet- a brief description
===
Network Architecture
---
Note:This section serves as a guide to what the module BPnetModel inside BPnet.jl implements with some slightly oversimplified examples and implementaions.
 
The overall network consists of several *blocks* (that we define as ConvBlock) consisting mainly of convolutional layers. Within a block, we have three stacks: left, right and main stacks. They process specific pieces of information to calculate $\widehat s_i$ for every $i$. Recall that the Ising model distribution has a resticted dependency field at every position consisting of spins to the left in the same row and spins to the right in the row above. The left stack attacks the spins to the left with convolutions while the right stack attacks the spins to the right in the row above with convolutions. The outputs from both left and right stacks are combined in the main stack by adding and acting with convolutions of filter size 1. We also perform the gating operation mentioned in [this](https://arxiv.org/abs/1606.05328) paper inside the main stack. A simple example is illustrated in the following figure for a 5x5 input configuration. Note how padding with zeros is performed to compensate for absence of spins to the left or top-right at the boundary.

![BPnet example](Figures/BPnet_sample.png) 

We'll construct a simple function that acts a ConvBlock similar to one in the above example on a given input configuration. First, we'll need to build a Julia [composite type](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) consisting of the various convolution layers we are going to use and declare it as a [Flux functor](https://fluxml.ai/Flux.jl/stable/models/functors/) so that the computer recognizes the trainable parameters present and provide functionalities for a possible GPU port:

In [ ]:
using Flux

struct ConvBlock
    l_conv::Flux.Conv
    r_conv::Flux.Conv
    m_conv1::Flux.Conv
    m_conv2::Flux.Conv
end

Flux.@functor ConvBlock

#Initializing a layer object
lc = Conv((1,2), 1=>1, tanh)
rc = Conv((1,2), 1=>1, tanh)
mc1 = Conv((1,1), 1=>2, relu)
mc2 = Conv((1,1), 1=>1, relu)

layer = ConvBlock(lc, rc, mc1, mc2)
#We instead define another method for ConvBlock in BPnet.jl
#so that inititializing doesn't require defining convolutional
#layers every time

Now, we'll define a function that acts the ConvBlock layer on an input. For zeropadding, we'll use the one we've defined inside the BPnet.jl file:

In [ ]:
include("BPnet.jl")
using BPnetModel:ZeroPad

function layer_call(layer, inp)
    x_l = inp[:,1:end-1,:,:]#cropping x_l
    x_r = inp[1:end-1, :, :, :]#cropping x_r
    x_r = ZeroPad(x_r, 1, 1, true)

    x_l = ZeroPad(x_l, 2, block.n - 1, true) #Padding left stack
    x_r = ZeroPad(x_r, 2, block.n - 1, false) #Padding right stack
    
    x_l = layer.l_conv(x_l) #Convolution on x_l, left stack
    x_r = layer.r_conv(x_r) #Convolution on x_r, right stack
    x_m = x_l .+ x_r #Combining information in main stack
    x_m = layer.m_conv(x_m)

    x_m0 = tanh.(x_m[:, :, 1:end÷2, :]) #splitting x_m across channels dim
    x_m1 = σ.(x_m[:, :, (end÷2)+1:end, :])
    x_m = x_m0 .* x_m1 #Gating on x_m, main stack

    out = layer.m_conv2(x_m) #Further conv on x_m
    return out
end

out = layer_call(layer, inp)

The actual BPnet model stacks several ConvBlock objects which increases its approximation power as well as expands the dependency field of $\widehat s_i$. The middle stack of the final block is acted upon by a (1,1) convolution with sigmoid activation and 1 out channel so that the output array of $\widehat s_i$ represents probabilities that fall inside the range (0,1). 

## Sampling and log probability calculations
The sample function of the BPnetModel model inside BPnet.jl facilitates sampling a lattice using the autoregressive network BPnet. Recall the definition of $\widehat s_i := q_{s_i=+1} = P(s_i=+1|s_{<i})$. Hence the sample function starts with a lattice of zeros of the desired lattice length and batch size. We loop over values of i from 1 to N. Since i=1 doesn't have any dependency, we can act the autoregressive network on this zeros lattice and use the output of $\widehat s_i$ for i=1 on a Bernoulli random number generator of $\pm 1$ values to obtain $s_1$. Now that we have $s_1$, in the next loop iteration we can look at the output for $\widehat s_2$ and predict $s_2$ from a Bernoulli random generator again. We recursively repeat this for i=3,4 and so on. Note that if we were using a naive PixelCNN network, we'll have to act the network over the entire lattice at every iteration, hence the overall time incurred by the sample function is $O(N^2)$. Since BPnet uses information only from the restricted dependency field of the Ising model, the sample function slices two rows and columns within the "range of reach" of the convolutional weight matrices corresponding to the ith position in the network. This code snippet does exactly that within the loop of the sample function:
```julia
x_l = sample[max(i-1,1):i, max(j-r, 1):min(j+r, L), :, :]
```
Note that the network is "size invariant", hence is able to act on this smaller array slice. Since this sliced array has no dependence on N it's computed in $O(1)$ time which means the overall computational time for the sample function for BPnet is just $O(N)$.

Though we gave an impression that the goal of autoregressive networks is to calculate the approximation $q_\theta$ to the Boltzmann distribution $p_B$, the order of magnitudes for this quantity for typical configurations is rather extreme especially for large system sizes. Also multiplying many conditional probabilities in the expression for $q_\theta$ can become numerically unstable if the terms are close to zero. Hence we've implemented a function log_prob that calculates the logarithm of $q_\theta$ by simply adding the logarithms of conditional probabilities $P(s_i|s_<i)$.

We've worked with h=0 set in $H_\text{ising}$ and hence imposed the Z2 symmetry by force-setting $\widehat s_1$=0.5, randomly multiplying the output lattices of the sample function with -1, as well as averaging the log probability of the given sample with that of it's inverted version in the log_prob function.